In [35]:
from sentinelhub import SHConfig

config = SHConfig()
config.sh_client_id = 'sh-fe080851-d462-4304-9f4d-1b7fd7c7dc95'
config.sh_client_secret = 'XBnxUPfjs1duw9wmexDpoVmhTbY6piHw'
config.sh_base_url = 'https://sh.dataspace.copernicus.eu'
config.sh_token_url = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

In [36]:
import datetime
import os
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import numpy as np
import rasterio
from rasterio.transform import from_bounds
from pathlib import Path
from sentinelhub import (
    CRS,
    BBox,
    DataCollection,
    DownloadRequest,
    MimeType,
    MosaickingOrder,
    SentinelHubDownloadClient,
    SentinelHubRequest,
    bbox_to_dimensions,
)


In [37]:
hyd_coords_wgs84 = (78.00405826,16.93264351,79.04971836,17.90150706)
resolution=(500,500)
hyd_bbox = BBox(bbox=hyd_coords_wgs84, crs=CRS.WGS84)
hyd_size = bbox_to_dimensions(hyd_bbox, resolution=resolution)

print(f"Image shape at {resolution} m resolution: {hyd_size} pixels")

Image shape at (500, 500) m resolution: (225, 212) pixels


In [38]:
evalscript_mean_mosaic = """
//VERSION=3
function setup() {
    return {
        input: ["NO2", "dataMask"],
        output: {
            bands: 1,
            sampleType: "FLOAT32",
        },
        mosaicking: "ORBIT"
    };
}

function isClear(sample) {
    return sample.dataMask == 1;
}

function sum(array) {
    let sum = 0;
    for (let i = 0; i < array.length; i++) {
        sum += array[i].NO2;
    }
    return sum;
}

function evaluatePixel(samples) {
    const clearTs = samples.filter(isClear)
    const mean = sum(clearTs) / clearTs.length
    return [mean]
}
"""

In [39]:
def get_data(time_interval):
    request = SentinelHubRequest(
        evalscript=evalscript_mean_mosaic,
        input_data=[
            SentinelHubRequest.input_data(
                data_collection=DataCollection.SENTINEL5P.define_from(
                    "s5pl2", service_url=config.sh_base_url
                ),
                time_interval=time_interval,
            )
        ],
        responses=[SentinelHubRequest.output_response("default", MimeType.TIFF)],
        bbox=hyd_bbox,
        size=hyd_size,
        config=config,
        data_folder=r"C:\Users\rishi\ml_projects\climate\temp_18_a\NO2",
    )
    raw_data = request.get_data(save_data=True)
    return raw_data[0]

In [40]:
get_data(("2018-12-1", "2018-12-31"))

array([[3.6480971e-05, 3.5828907e-05, 3.5310404e-05, ..., 3.4005938e-05,
        3.4663906e-05, 3.4409997e-05],
       [3.6480971e-05, 3.7684185e-05, 3.6367721e-05, ..., 3.3432967e-05,
        3.3586108e-05, 3.4425364e-05],
       [3.7293245e-05, 3.7791207e-05, 3.7511098e-05, ..., 3.2493139e-05,
        3.1940879e-05, 3.1427695e-05],
       ...,
       [3.0869593e-05, 3.0638446e-05, 3.1086987e-05, ..., 3.1709918e-05,
        3.2349202e-05, 3.2144952e-05],
       [3.0869593e-05, 3.1533647e-05, 3.1086987e-05, ..., 3.2441501e-05,
        3.1959262e-05, 3.1689502e-05],
       [3.0959603e-05, 3.1871310e-05, 3.0976109e-05, ..., 3.0776231e-05,
        3.1769661e-05, 3.1968568e-05]], dtype=float32)

In [41]:
# months = {
#     1: ("January", 31),
#     2: ("February", 28),
#     3: ("March", 31),
#     4: ("April", 30),
#     5: ("May", 31),
#     6: ("June", 30),
#     7: ("July", 31),
#     8: ("August", 31),
#     9: ("September", 30),
#     10: ("October", 31),
#     11: ("November", 30),
#     12: ("December", 31)
# }
# vals=[]
# output_dir=r"C:\Users\rishi\ml_projects\climate\2023_data\NO2"
# for i in range(5,8):
#     month_name, end_day = months[i]
#     start=f"2023-{i}-1"
#     end = f"2023-{i}-{end_day}"
#     time_interval=(start,end)
#     arr=get_data(time_interval)
#     path=os.path.join(output_dir, f'{month_name}_NO2.csv')
#     np.savetxt(path, arr, delimiter=',')

In [42]:
# months = {
#     1: ("January", 31),
#     2: ("February", 28),
#     3: ("March", 31),
#     4: ("April", 30),
#     5: ("May", 31),
#     6: ("June", 30),
#     7: ("July", 31),
#     8: ("August", 31),
#     9: ("September", 30),
#     10: ("October", 31),
#     11: ("November", 30),
#     12: ("December", 31)
# }
# vals=[]
# output_dir=r"C:\Users\rishi\ml_projects\climate\2023_data\NO2"
# for i in range(5,8):
#     month_name, end_day = months[i]
#     start=f"2023-{i}-1"
#     end = f"2023-{i}-{end_day}"
#     time_interval=(start,end)
#     arr=get_data(time_interval)
#     path=os.path.join(output_dir, f'{month_name}_NO2.csv')
#     np.savetxt(path, arr, delimiter=',')

In [43]:
import geopandas as gpd

# Load the shapefile
shapefile_path = r'C:\Users\rishi\ml_projects\climate\hmda_boundary\hmda_boundary.shp'
gdf = gpd.read_file(shapefile_path)

import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
# Load the GeoTIFF file
vmin = .1072  # Minimum value for normalization
vmax = .1086  # Maximum value for normalization

# Open the GeoTIFF file
tiff_file = r'C:\Users\rishi\ml_projects\climate\temp_18\NO2\a8348adcc9b838d21003a51523c6fe0b\response.tiff'
with rasterio.open(tiff_file) as dataset:
    # Create a normalization instance
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    
    # Display the image with geospatial context
    fig, ax = plt.subplots(figsize=(10, 10))
    show(dataset, ax=ax, cmap='rainbow')
    plt.title('GeoTIFF Data Visualization')
    
    # Add a colorbar with the same normalization
    plt.colorbar(ax.images[0], ax=ax, orientation='vertical')
    
    # Plot the boundary of the GeoDataFrame
    gdf.boundary.plot(ax=ax, edgecolor='black')
    
    plt.show()

RasterioIOError: C:/Users/rishi/ml_projects/climate/temp_18/NO2/a8348adcc9b838d21003a51523c6fe0b/response.tiff: No such file or directory

In [19]:
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Rectangle

# Load the shapefile
shapefile_path = r'C:\Users\rishi\ml_projects\climate\hmda_boundary\hmda_boundary.shp'
gdf = gpd.read_file(shapefile_path)

# Load the GeoTIFF file
vmin = .1072  # Minimum value for normalization
vmax = .1086  # Maximum value for normalization

# Define the bounding box coordinates (minx, miny, maxx, maxy)
bbox_coords = [78.39962, 17.320, 78.5922, 17.482]  # Replace with actual coordinates

# Open the GeoTIFF file(78.00405826,16.93264351,79.04971836,17.90150706)
tiff_file = r'C:\Users\rishi\ml_projects\climate\Notebooks\data\no2\f977e6499a361aef51600fe76d79be8d\response.tiff'
with rasterio.open(tiff_file) as dataset:
    # Create a normalization instance
    norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
    
    # Display the image with geospatial context
    fig, ax = plt.subplots(figsize=(10, 10))
    show(dataset, ax=ax, cmap='rainbow')
    plt.title('GeoTIFF Data Visualization')
    
    # Add a colorbar with the same normalization
    plt.colorbar(ax.images[0], ax=ax, orientation='vertical')
    
    # Plot the boundary of the GeoDataFrame
    gdf.boundary.plot(ax=ax, edgecolor='black')
    
    # Add the bounding box in white
    bbox = Rectangle((bbox_coords[0], bbox_coords[1]),  # Lower-left corner
                     bbox_coords[2] - bbox_coords[0],  # Width
                     bbox_coords[3] - bbox_coords[1],  # Height
                     linewidth=3, edgecolor='black', facecolor='black')
    ax.add_patch(bbox)
    
    plt.show()


RasterioIOError: C:/Users/rishi/ml_projects/climate/Notebooks/data/no2/f977e6499a361aef51600fe76d79be8d/response.tiff: No such file or directory

In [12]:
with rasterio.open(tiff_file) as dataset:
    # Print metadata
    print(dataset.meta)
    num_bands = dataset.count
    print(f"The GeoTIFF file has {num_bands} band(s).")
    # Get the spatial extent (bounding box)
    print("Bounds:", dataset.bounds)
    
    # Read the data as a numpy array (e.g., first band)
    data = dataset.read(1)
    print("Data shape:", data.shape)
    print(data[:5][:5])

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 225, 'height': 212, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.00464737822222225, 0.0, 78.00405826,
       0.0, -0.004570111084905669, 17.90150706)}
The GeoTIFF file has 1 band(s).
Bounds: BoundingBox(left=78.00405826, bottom=16.93264351, right=79.04971836, top=17.90150706)
Data shape: (212, 225)
[[2.1347754e-05 1.8042352e-05 1.9794054e-05 ... 1.9593117e-05
  2.3457347e-05 2.3457347e-05]
 [2.1347754e-05 2.1117701e-05 2.3638238e-05 ... 1.9593117e-05
  2.2249007e-05 2.0876305e-05]
 [2.1347754e-05 2.1117701e-05 2.3638238e-05 ... 1.8740398e-05
  2.0018191e-05 2.0018191e-05]
 [2.1347754e-05 2.1117701e-05 2.1117701e-05 ... 1.7648119e-05
  1.7984414e-05 1.7984414e-05]
 [2.2105642e-05 2.2105642e-05 2.1628603e-05 ... 1.8055514e-05
  1.6706621e-05 1.7984414e-05]]
